This is the **second script to run** in the workflow.  

# Build EU Transposition Dataset for Selected Directives

This script consolidates national transposition data for a set of Green Deal–related directives into a single cleaned dataset.  

**Steps:**  
1. Load national transposition tables for selected directives.  
2. Keep relevant variables (directive ID, country, dates, amending flag, subject).  
3. Concatenate all directives into one dataset.  
4. Drop rows with missing/invalid country codes and exclude the UK.  
5. Convert date variables to datetime format.  
6. Standardize "Amending" as a boolean.  
7. Remove incoherent rows where publication date is earlier than date of effect.  

In [ ]:
import pandas as pd
import numpy as np

# === Step 1: Load input datasets ===
# Load national transposition tables for selected Green Deal-related directives.
# ! Replace "insert/your/path" with the actual location of your CSV files.
df_32017L2001 = pd.read_csv("insert/your/path/transposition_32018L2001.csv")
df_32004L0101 = pd.read_csv("insert/your/path/transposition_32004L0101.csv")
df_32009L0029 = pd.read_csv("insert/your/path/transposition_32009L0029.csv")
df_32010L0031 = pd.read_csv("insert/your/path/transposition_32010L0031.csv")
df_32018L0410 = pd.read_csv("insert/your/path/transposition_32018L0410.csv")
df_32018L0844 = pd.read_csv("insert/your/path/transposition_32018L0844.csv")
df_32018L2002 = pd.read_csv("insert/your/path/transposition_32018L2002.csv")
df_32023L0958 = pd.read_csv("insert/your/path/transposition_32023L0958.csv")
df_32023L0959 = pd.read_csv("insert/your/path/transposition_32023L0959.csv")
df_32023L1791 = pd.read_csv("insert/your/path/transposition_32023L1791.csv")
df_32023L2413 = pd.read_csv("insert/your/path/transposition_32023L2413.csv")
df_32024L1275 = pd.read_csv("insert/your/path/transposition_32024L1275.csv")
df_32009L0028 = pd.read_csv("insert/your/path/transposition_32009L0028.csv")

# === Step 2: Select relevant variables ===
cols_to_keep = [
    "CELEX",
    "Directive name",
    "Date of document",
    "Date of effect",
    "Amending",
    "Country Code",
    "Country",
    "Measure Title",
    "Transposition Deadline",
    "Publication Date",
    "Subject matter"
]

# Apply selection
df_32017L2001_clean = df_32017L2001[cols_to_keep].copy()
df_32004L0101_clean = df_32004L0101[cols_to_keep].copy()
df_32009L0029_clean = df_32009L0029[cols_to_keep].copy()
df_32010L0031_clean = df_32010L0031[cols_to_keep].copy()
df_32018L0410_clean = df_32018L0410[cols_to_keep].copy()
df_32018L0844_clean = df_32018L0844[cols_to_keep].copy()
df_32018L2002_clean = df_32018L2002[cols_to_keep].copy()
df_32023L0958_clean = df_32023L0958[cols_to_keep].copy()
df_32023L0959_clean = df_32023L0959[cols_to_keep].copy()
df_32023L1791_clean = df_32023L1791[cols_to_keep].copy()
df_32023L2413_clean = df_32023L2413[cols_to_keep].copy()
df_32024L1275_clean = df_32024L1275[cols_to_keep].copy()
df_32009L0028_clean = df_32009L0028[cols_to_keep].copy()

# === Step 3: Concatenate datasets ===
dfs = [
    df_32017L2001_clean,
    df_32004L0101_clean,
    df_32009L0029_clean,
    df_32010L0031_clean,
    df_32018L0410_clean,
    df_32018L0844_clean,
    df_32018L2002_clean,
    df_32023L0958_clean,
    df_32023L0959_clean,
    df_32023L1791_clean,
    df_32023L2413_clean,
    df_32024L1275_clean,
    df_32009L0028_clean
]
df_all = pd.concat(dfs, ignore_index=True)

# === Step 4: Filter by geography (EU only) ===
df_all = df_all[
    (df_all["Country Code"].notna()) &
    (df_all["Country Code"] != "GBR") &
    (df_all["Country Code"].str.strip() != "")
]

# === Step 5: Convert date variables ===
date_cols = [
    "Date of document",
    "Date of effect",
    "Transposition Deadline",
    "Publication Date",
    "Date of transposition"
]
for col in date_cols:
    if col in df_all.columns:
        df_all[col] = pd.to_datetime(df_all[col], format="%m/%d/%y", errors="coerce")

# === Step 6: Clean boolean variable ===
df_all["Amending"] = (
    df_all["Amending"]
    .astype(str)
    .str.strip()
    .str.lower()
    .map({"true": True, "false": False, "yes": True, "no": False, "1": True, "0": False})
)

# === Step 7: Detect and drop incoherent rows ===
incoherent_dates = df_all["Publication Date"] < df_all["Date of effect"]
df_all = df_all[~incoherent_dates]

print("Final dataset dimensions:", df_all.shape)

This is the **third script to run** in the workflow.  

# Create Dependent Variable: Compliance Time

This script calculates the time each Member State took to transpose EU directives, measured as the delay or advance relative to the official deadline.  

**Steps:**  
1. Compute compliance time as days, weeks, and months between *Transposition Deadline* and *Publication Date*.  
2. Check results with a preview of compliance variables.  
3. Save the final dataset as `gd_ds_transposition.csv`.  

In [ ]:
# === Step 1: Create dependent variable (compliance time) ===
# Calculate time between the transposition deadline and the actual publication date.

# Difference in days
df_all["Compliance_days"] = (
    (df_all["Publication Date"] - df_all["Transposition Deadline"]).dt.days
)

# Difference in weeks
df_all["Compliance_weeks"] = df_all["Compliance_days"] / 7

# Difference in months (approximation: 30 days per month)
df_all["Compliance_months"] = df_all["Compliance_days"] / 30

# === Step 2: Check results ===
# Display CELEX, country, deadlines, publication dates, and compliance variables.
print(
    df_all[
        [
            "CELEX",
            "Country Code",
            "Transposition Deadline",
            "Publication Date",
            "Compliance_days",
            "Compliance_weeks",
            "Compliance_months"
        ]
    ].head(50)
)

# === Step 3: Save dataset ===
# ! Replace "insert/your/path" with your actual folder.
df_all.to_csv("insert/your/path/gd_ds_transposition.csv", index=False)
print("Dataset saved successfully.")


This is the **sixth script to run** in the workflow.  

**⚠️ Important prerequisite:**
Before running this script, you must have already executed media_salience_rcm.iynp, which generates the **MediaSalience_*.csv** files used here.

# Merge Media Salience Data with EU Directive Dataset

This script merges **Media Cloud salience** indicators with the dataset of EU Green Deal directives.
It attaches media salience measures to each directive based on country and publication year.


This is the **seventh script to run** in the workflow.  

**⚠️ Important prerequisite:**
Before running this script, you must have already executed media_salience_rcm.iynp, which generates the **RCM_*.csv** files used here.

# Merge RCM Data with EU Directive Dataset

This script merges **Relative Climate Salience (RCM)** indicators (from Google Trends) with the dataset of EU Green Deal directives.
It attaches RCM values to each directive based on country and publication year.


In [ ]:
import pandas as pd
import os

def merge_rcm_with_transposition(transposition_file, rcm_folder, output_file):
    """
    Merge Relative Climate Salience (RCM) indicators from Google Trends 
    with the EU directives dataset.
    """

    # === 1. Load transposition dataset ===
    df_trans = pd.read_csv(transposition_file)

    # === 2. Ensure 'Publication Date' is datetime and extract year ===
    df_trans['Publication Date'] = pd.to_datetime(df_trans['Publication Date'], errors='coerce')
    df_trans['year'] = df_trans['Publication Date'].dt.year

    # === 3. Initialize empty RCM columns ===
    df_trans['Climate salience (GT)'] = pd.NA
    df_trans['Economy salience (GT)'] = pd.NA
    df_trans['Relative Climate salience (RCM)'] = pd.NA

    # === 4. Loop through each RCM file in the folder ===
    for file_name in os.listdir(rcm_folder):
        if file_name.startswith("RCM_") and file_name.endswith(".csv"):
            path = os.path.join(rcm_folder, file_name)
            try:
                # Load one country’s RCM dataset
                df_rcm = pd.read_csv(path)

                # Extract country metadata
                country_code = df_rcm['Country Code'].iloc[0]
                country_name = df_rcm['Country'].iloc[0]

                # === 5. Merge RCM data by year and country ===
                for idx, row in df_rcm.iterrows():
                    mask = (
                        (df_trans['Country Code'] == country_code) &
                        (df_trans['Country'] == country_name) &
                        (df_trans['year'] == row['year'])
                    )
                    df_trans.loc[mask, 'Climate salience (GT)'] = row['Climate salience (GT)']
                    df_trans.loc[mask, 'Economy salience (GT)'] = row['Economy salience (GT)']
                    df_trans.loc[mask, 'Relative Climate salience (RCM)'] = row['Relative Climate salience (RCM)']

                print(f"Merged RCM for {country_name}")
            except Exception as e:
                print(f"Error with {file_name}: {e}")

    # === 6. Save merged dataset ===
    df_trans.to_csv(output_file, index=False)
    print(f"Final file saved to: {output_file}")


# === Example usage ===
if __name__ == "__main__":
    #Prerequisite: run rcm_by_country.iynp first to generate RCM_*.csv files
    transposition_csv = 'insert/your/path/gd_ds_transposition_with_mediacloud.csv'
    rcm_folder = 'insert/your/path/RCM by country'
    output_csv = 'insert/your/path/gd_ds_transposition_mediacloud_rcm.csv'

    merge_rcm_with_transposition(transposition_csv, rcm_folder, output_csv)

## ⚠️ READ CAREFULLY BELOW ⚠️
---
### **Prerequisites**  
Normally, you would need to run the notebook **`datagov_manifesto.iynb`** to generate the government–manifesto merged dataset.  
- ⚠️ The raw **DataGov dataset** was incomplete (many governments were missing).  
- These missing governments had to be **added manually**.  
- For reproducibility, the cleaned and aggregated dataset `aggregated_governments_with_weighted_avg.csv` has been made available on **GitHub** at:   
  ```
   --> insert path of github
  ```  
---

This is the **thirtheenth script to run** in the workflow.  

# Match EU Directives with Governments in Power  

This script matches each **EU Green Deal directive** with the **government in power at the time of its publication**.  
It merges the directives dataset (with compliance and salience indicators) with the governments dataset (with weighted averages of party positions).  




In [ ]:
import pandas as pd

# === 1. Load datasets ===
transposition_df = pd.read_csv('insert/your/path/gd_ds_transposition_mediacloud_rcm.csv')
govs_df = pd.read_csv('insert/your/path/aggregated_governments_with_weighted_avg.csv')

# === 2. Convert date variables to datetime ===
transposition_df["Publication Date"] = pd.to_datetime(transposition_df["Publication Date"], errors="coerce")
govs_df["Start Date"] = pd.to_datetime(govs_df["Start Date"], errors="coerce")

# === 3. Sort governments by country and start date ===
govs_df = govs_df.sort_values(by=["Country Code", "Start Date"])

# === 4. Function: find government in power at publication date ===
def find_government(row, govs):
    country = row["Country Code"]
    pub_date = row["Publication Date"]

    # Filter governments in same country with start date <= publication date
    govs_in_country = govs[(govs["Country Code"] == country) & (govs["Start Date"] <= pub_date)]

    if not govs_in_country.empty:
        # Take the latest (most recent) government
        return govs_in_country.iloc[-1]
    else:
        return pd.Series()  # No government found (edge case)

# === 5. Apply function across directives ===
matched_governments = transposition_df.apply(lambda row: find_government(row, govs_df), axis=1)

# === 6. Combine directive data with matched government attributes ===
combined_df = pd.concat([transposition_df.reset_index(drop=True), matched_governments.reset_index(drop=True)], axis=1)

# === 7. Save final merged dataset ===
combined_df.to_csv('insert/your/path/gd_ds_transposition_with_govs.csv', index=False)

This is the **fourteenth script to run** in the workflow.  

# Add Infringement and Short Names to Directive–Government Dataset  

This script prepares the **directives–government merged dataset** for further analysis (e.g., in Stata).  
Because Stata has difficulty handling long string variables, the script generates **shortened versions** of measure titles and directive names.  
It also flags infringement procedures and extracts references to amended directives.  


In [ ]:
import pandas as pd
import re

# === 1. Load dataset ===
df = pd.read_csv('insert/your/path/gd_ds_transposition_with_govs.csv')

# === 2. Create shortened version of "Measure Title" (for Stata compatibility) ===
measure_title_short = df["Measure Title"].str.slice(0, 32)

# Remove existing column if present (to avoid duplicates)
if "MeasureTitle_short" in df.columns:
    df.drop(columns=["MeasureTitle_short"], inplace=True)

# Insert new column right after "Country"
col_index_country = df.columns.get_loc("Country") + 1
df.insert(col_index_country, "MeasureTitle_short", measure_title_short)

# Drop original "Measure Title" column (keep only short version)
if "Measure Title" in df.columns:
    df.drop(columns=["Measure Title"], inplace=True)

# === 3. Create infringement-related variables ===
# Flag: True if measure title starts with "INFR"
infringement_procedure = measure_title_short.fillna("").str.startswith("INFR")

# Extract infringement name if flag is True
infringement_name = measure_title_short.where(infringement_procedure)

# Remove old columns if present
for col in ["Infringement_Procedure", "Infringement_Name"]:
    if col in df.columns:
        df.drop(columns=[col], inplace=True)

# Insert new infringement columns right after "Compliance_months"
col_index_comp = df.columns.get_loc("Compliance_months") + 1
df.insert(col_index_comp, "Infringement_Procedure", infringement_procedure.astype(bool))
df.insert(col_index_comp + 1, "Infringement_Name", infringement_name)

# === 4. Create short directive names ===
def extract_amended(text, amending_flag):
    """
    Extracts the name of the amended directive if 'Amending' is True
    and the directive name contains an 'amending Directive ...' clause.
    """
    if not amending_flag or pd.isna(text):
        return ""
    match = re.search(r"amending\s+(Directive\s+[A-Z]*\s*\d+/\d+)", text)
    if match:
        return match.group(1)
    return ""

# Extract base directive name (before " of ...")
df["DirectiveName_short"] = df["Directive name"].str.split(" of ", n=1).str[0]

# Extract amended directive reference if applicable
df["AmendedName_short"] = [
    extract_amended(txt, amend) for txt, amend in zip(df["Directive name"], df["Amending"])
]

# Insert short directive names right after "Directive name"
col_index_dir = df.columns.get_loc("Directive name") + 1
df.insert(col_index_dir, "DirectiveName_short", df.pop("DirectiveName_short"))
df.insert(col_index_dir + 1, "AmendedName_short", df.pop("AmendedName_short"))

# Drop original "Directive name"
if "Directive name" in df.columns:
    df.drop(columns=["Directive name"], inplace=True)

# === 5. Save updated dataset (UTF-8 BOM for Stata compatibility) ===
output_path = 'insert/your/path/gd_ds_transposition_with_govs_infr.csv'
df.to_csv(output_path, index=False, encoding="utf-8-sig")

print(f"File correctly saved in: {output_path}")

This is the **last script to run** in the workflow. 

--- 
## Prerequisites 

Before running this script, you must first run:  
**`EB_merger.iynb`** → generates the harmonised and merged Eurobarometer CRP dataset (`EB_CRPs_interp.csv`).  

---
# Merge Directives Dataset with Eurobarometer CRP  

This script merges the **EU directives–government dataset** with the **Eurobarometer climate risk perception (CRP) panel**.  

The output is saved as an **Excel file** (`.xlsx`), ready to be imported into **Stata**.  
Excel format was chosen because saving as `.csv` caused many numeric variables to be incorrectly recorded as strings, creating issues for analysis.  

In [ ]:
import pandas as pd

# === File paths ===
transposition_csv = "insert/your/path/gd_ds_transposition_with_govs_infr.csv"
eb_crps_csv = "insert/your/path/EB_CRPs_interp.csv"
output_csv = "insert/your/path/gd_ds_transposition_for_stata.csv"


def merge_transposition_with_ebcrps(transposition_file, eb_file, output_file):
    """
    Merge EU directives–government dataset with interpolated
    Eurobarometer climate risk perception (CRP) values and
    save the result as an Excel file.
    """

    # === 1. Load datasets ===
    df_trans = pd.read_csv(transposition_file)
    df_eb = pd.read_csv(eb_file)

    # === 2. Ensure 'year' exists in transposition data ===
    if "year" not in df_trans.columns and "Publication Date" in df_trans.columns:
        df_trans["Publication Date"] = pd.to_datetime(df_trans["Publication Date"], errors="coerce")
        df_trans["year"] = df_trans["Publication Date"].dt.year

    # === 3. Merge on country and year ===
    df_merged = df_trans.merge(
        df_eb,
        left_on=["Country", "year"],
        right_on=["country", "year"],
        how="left"
    )

    # === 4. Drop duplicate 'country' column from EB_CRPs ===
    if "country" in df_merged.columns:
        df_merged.drop(columns=["country"], inplace=True)

    # === 5. Check for missing matches ===
    missing = df_merged[df_merged["crp_raw"].isna()]
    print(f" Missing matches: {len(missing)} out of {len(df_merged)} rows")

    # === 6. Save merged dataset as Excel ===
    df_merged.to_excel(output_file, index=False, engine="openpyxl")
    print(f"Final file saved to: {output_file}")